In [1]:
import coffea
from git import Repo
import awkward as ak
import numpy as np
import numba
import uproot
import dask_awkward as dak
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")

_______________________________________
	Current Configuration
---------------------------------------
Coffea Version:  0.1.dev3583+ge06c4b8
Branch: 	 master
Modified by: 	 Prayag Yadav
_______________________________________


In [2]:
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
test_file = 'root://eospublic.cern.ch//eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/p8_ee_ZH_ecm240/events_101027117.root'
#test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
file = uproot.open(test_file)

events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass= FCC.get_schema(version="latest"),
    delayed = False,
    metadata=file["metadata"].arrays()
).events()

file.close()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/fcc.py:4: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


# 1. MCRecoAssociations

- The relationship betweeen the ReconstructedParticles and their generated MC counterparts are saved in MCRecoAssociations collection.
- Using BaseSchema one finds that there are 5 braches associated with MCRecoAssociations:
    - `MCRecoAssociations/MCRecoAssociations.weight`
    - `MCRecoAssociations#0/MCRecoAssociations#0.index`
    - `MCRecoAssociations#0/MCRecoAssociations#0.collectionID`
    - `MCRecoAssociations#1/MCRecoAssociations#1.index`
    - `MCRecoAssociations#1/MCRecoAssociations#1.collectionID`
- `MCRecoAssociations#1` have indices corresponding to `ReconstructedParticles` collection (same collectionIDs)
- `MCRecoAssociations#0` have indices corresponding to `Particle` collection which is the MC collection (same collectionIDs)

- I have zipped these branches to produce the `MCRecoAssociations` collection with the following structure:

In [3]:
def tree(Record_Array):
    '''
    To list down all the fields and subfields of a ListOffsetArray with RecordArray members
    '''
    f,l,d = ak.to_buffers(Record_Array)
    f = f.to_dict()
    if f["class"] == "ListOffsetArray":
        c = f["content"]
    elif f["class"] == "RecordArray":
        c = f["contents"]
    else :
        raise TypeError("Incompatible class type of the input array.")
    if c["class"] == "RecordArray":
        fields = c["fields"]
        for num, field in enumerate(fields):
            print("*", field)
            s = c["contents"][num]
            if s["class"] == "RecordArray":
                sub_fields = s["fields"]
                for sub_field in sub_fields:
                    print("\t-",sub_field)
                print()
        print("\n")

In [4]:
tree(events.MCRecoAssociations)

* mc
	- collectionID
	- index

* reco
	- collectionID
	- index

* weight




In [5]:
events.MCRecoAssociations.weight

<Array [[0, 0, 0, 0, 0, ..., 0, 0, 0, 0], ...] type='100 * var * float32[pa...'>

In [6]:
events.MCRecoAssociations.mc.fields

['collectionID', 'index']

In [7]:
events.MCRecoAssociations.reco.fields

['collectionID', 'index']

In [8]:
events.MCRecoAssociations.mc.index

<Array [[174, 178, 138, ..., 267, 168], ...] type='100 * var * int32[parame...'>

In [9]:
events.MCRecoAssociations.mc.collectionID

<Array [[5, 5, 5, 5, 5, ..., 5, 5, 5, 5], ...] type='100 * var * int32[para...'>

- I have defined a new mixin class for the `MCRecoAssociations` collection, called `ParticleLink`, to facilitate the definition of some useful properties

---
```python
@awkward.mixin_class(behavior)
class ParticleLink(base.NanoCollection):
    """MCRecoParticleAssociation objects."""

    @property
    def reco_mc_index(self):
        """
        Returns an array of indices mapping to generator particles for each reconstructed particle
        """
        arr_reco = self.reco.index[:,:,numpy.newaxis]
        arr_mc = self.mc.index[:,:,numpy.newaxis]

        joined_indices = awkward.concatenate((arr_reco,arr_mc), axis=2)

        return joined_indices

    @dask_property
    def reco_mc(self):
        """
        Returns an array of Records mapping to generator particle record for each reconstructed particle record
        """
        reco_index = self.reco.index
        mc_index = self.mc.index
        r = self._events().ReconstructedParticles[reco_index][:,:,numpy.newaxis]
        m = self._events().Particle[mc_index][:,:,numpy.newaxis]

        return awkward.concatenate((r,m), axis=2)

    @reco_mc.dask
    def reco_mc(self, dask_array):
        """
        Returns an array of Records mapping to generator particle record for each reconstructed particle record
        """
        reco_index = dask_array.reco.index
        mc_index = dask_array.mc.index
        r = dask_array._events().ReconstructedParticles[reco_index][:,:,numpy.newaxis]
        m = dask_array._events().Particle[mc_index][:,:,numpy.newaxis]

        return awkward.concatenate((r,m), axis=2)

```
---

## Lets see these functions in action

- `reco_mc_index` creates the MC index for each Reco index

In [10]:
events.MCRecoAssociations.reco_mc_index

<Array [[[0, 174], [1, ...], ..., [80, 168]], ...] type='100 * var * 2 * int32'>

In [11]:
# The first index association in the first event
# 0th ReconstructedParticle in first event corresponds to the 88th MC Particle in the Particle collection
events.MCRecoAssociations.reco_mc_index[0,0,:]

<Array [0, 174] type='2 * int32'>

- To get the actual data instead of indices from `reco_mc_index`, we simply use `reco_mc`

In [12]:
print(events.MCRecoAssociations.reco_mc)

[[[RecoParticle, MCTruthParticle], ..., [RecoParticle, ...]], ...]


In [13]:
# The first association in the first event
# 0th ReconstructedParticle in first event corresponds to the 88th MC Particle in the Particle collection
events.MCRecoAssociations.reco_mc[0,0,:]

<Array [RecoParticle, MCTruthParticle] type='2 * union[RecoParticle[E: floa...'>

In [14]:
reco = events.MCRecoAssociations.reco_mc[0,0,0]
mc = events.MCRecoAssociations.reco_mc[0,0,1]

In [15]:
reco.charge

np.float32(1.0)

In [16]:
mc.PDG

np.int32(211)

- -211 pdg id might be $\pi^-$

In [17]:
reco["mass"]

np.float32(0.13957039)

In [18]:
mc["mass"]

np.float64(0.13956999778747559)

## Modifications to RecoParticle Class

- To facilitate easy access, one can also get the MC collection corresponding to a ReconstructedParticle

In [19]:
events.ReconstructedParticles.matched_gen

<MCTruthParticleArray [[MCTruthParticle, ..., MCTruthParticle], ...] type='...'>

# 2. A more difficult problem : get daughter/parent particles

- Each of the MC particles in the `Particle` collection has varrying number of daughters
- `Particles.daughters.begin`and `Particles.daughters.end` define the range of indices in the branch `Particleidx1` that corresponds to the daughter particles of the specific particle.
- Similarly, `Particles.parents.begin`and `Particles.parents.end` define the range of indices in the branch `Particleidx0` that corresponds to the parent particles of the specific particle.

In [20]:
events.Particle.daughters.fields

['begin', 'end']

In [21]:
events.Particle.daughters.begin

<Array [[0, 1, 2, 4, ..., 576, 576, 576], ...] type='100 * var * uint32[par...'>

In [22]:
events.Particle.daughters.end

<Array [[1, 2, 4, 6, ..., 576, 576, 576], ...] type='100 * var * uint32[par...'>

In [23]:
n_daughters = events.Particle.daughters.end - events.Particle.daughters.begin
n_daughters

<Array [[1, 1, 2, 2, 2, 2, ..., 0, 0, 0, 0, 0], ...] type='100 * var * uint32'>

- As an example, lets find the daughter particles of the first MC particle in the first event

In [24]:
indices = list(range(events.Particle.daughters.begin[0,0],events.Particle.daughters.end[0,0]))
daughter_indices = events.Particleidx1.index[0][indices]
daughter_indices

<Array [2] type='1 * int32[parameters={"__doc__": "index[Particle#1_]"}]'>

- [2] is one membered and means that the First MC Particle in the first event has only one daughter and that daughter is present in the 2nd index of the `Particle` collection
- To get the actual daughter MC particle:

In [25]:
events.Particle[0][daughter_indices]

<MCTruthParticleArray [{PDG: 11, charge: -1, ...}] type='1 * MCTruthParticl...'>

- We have to do this whole process for each particle in each event for all the events!
- I could not use numba just-in-time compilation because its too complex to build and the numba function may not work in delayed mode

- What I aim to get is an array which is doubly nested, that contains all the daughters corresponding to all the particles
- It might look something like this:
---
```
[ [ [daughter_0, daughter_1, ..., daughter_n], [daughter_0, daughter_1, ..., daughter_m], ... [] ],
  [ [daughter_0, daughter_1, ..., daughter_o], [daughter_0, daughter_1, ..., daughter_p], ... [] ],
  [ [daughter_0, daughter_1, ..., daughter_q], [daughter_0, daughter_1, ..., daughter_r], ... [] ],
  .
  .
  . 
  [ ]
]
type: 100 * var * var * MCTruthParticle[...]
```
---

- So far, I could not come up with a way to achieve this result with a reasonable time complexity.
- I would like to hear suggestions on how to do this. 